## **Imports**

In [ ]:
import os
import PIL
import cv2
import tarfile
import numpy as np
import pydicom
import pandas as pd
from glob import glob
import nibabel as nib
from PIL import Image
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

## **Extract the Tar Files Containing the Task 1 Dataset**

In [ ]:
# Create Data Directory
if not os.path.isdir("/kaggle/working/data"):
    os.makedirs("/kaggle/working/data", exist_ok=True)
    
# Load Competition Training Dataframe
train_df = pd.read_csv("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")

# Extract Main Training Data
print("... Extracting Main Task1 Training Files (3-5 Minutes) ...\n")
tar = tarfile.open("/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar")
tar.extractall("/kaggle/working/data")
tar.close()

# Extract Update
print("\n... Extracting BraTSID=00495 Task1 Update Files ...")
tar = tarfile.open("/kaggle/input/brats-2021-task1/BraTS2021_00495.tar")
tar.extractall("/kaggle/working/data")
tar.close()

# Extract Update
print("... Extracting BraTSID=00621 Task1 Update Files ...")
tar = tarfile.open("/kaggle/input/brats-2021-task1/BraTS2021_00621.tar")
tar.extractall("/kaggle/working/data")
tar.close()

## **Plot An Assortment of Task 1 Dataset Images At A Particular Scan Depth**

In [ ]:
img_ids = ["00376", "00789", "00441", "00703", "00807", "00523", "00241", "00778",]
SCAN_NUM = 83

for img_id in img_ids:
    print(f"\n\n\n... IMAGE ID={img_id} **MGMT={train_df[train_df['BraTS21ID']==int(img_id)].MGMT_value.values[0]}** ...       [SHOWING SLICE/SCAN {SCAN_NUM}] ")
    plt.figure(figsize=(18, 5))

    for i, nii in enumerate([f'./data/BraTS2021_{img_id}/BraTS2021_{img_id}_{s_type}.nii.gz' for s_type in ["flair", "t1", "t1ce", "t2", "seg"]]):
        # PLOTTING
        plt.subplot(1,5,i+1)
        image = nib.load(nii).get_fdata()
        plt.title(nii.rsplit("_", 1)[1].split(".", 1)[0], fontweight="bold")
        plt.axis(False)
        plt.imshow(image[:, :, SCAN_NUM], cmap="bone")
        
    plt.tight_layout()    
    plt.show()

## **All Scans For Patient <font color="blue">#441</font> - <font color="red">Pulling From Task 1 Dataset</font>**

In [ ]:
img_id = "00441"


for i, nii in enumerate([f'./data/BraTS2021_{img_id}/BraTS2021_{img_id}_{s_type}.nii.gz' for s_type in ["flair", "t1", "t1ce", "t2", "seg"]]):
    # PLOTTING
    image = nib.load(nii).get_fdata()
    slices = image.shape[-1]
    rows = int(np.ceil((slices/2)/10))
    plt.figure(figsize=(20, rows*2))
    plt.suptitle(f"\n\n\n{nii.rsplit('_', 1)[-1].split('.', 1)[0]} SCAN\n".upper(), fontsize=18, fontweight="bold")
    for j in range(0, slices, 2):
        plt.subplot(rows, 10, 1+j//2)
        plt.axis(False)
        plt.imshow(image[:, :, j], cmap="bone")
    plt.tight_layout()
    plt.show()

## **All Scans For Patient <font color="blue">#441</font> - <font color="red">Pulling From Task 2 Dataset</font>**

In [ ]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

for i, scan_type in enumerate(["FLAIR", "T1w", "T1wCE", "T2w"]):
    dicom_paths = [os.path.join("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train", img_id, scan_type, x) for x in os.listdir(os.path.join("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train", img_id, scan_type))]
    dicom_paths = sorted(dicom_paths, key=lambda x: (int(x.rsplit("-", 1)[-1].split(".", 1)[0])), reverse=True)
    slices = max([int(x.rsplit("-", 1)[-1].split(".", 1)[0]) for x in dicom_paths])
    rows = int(np.ceil(slices/8))
    plt.figure(figsize=(20, rows*4))
    plt.suptitle(f"\n\n\n{scan_type} SCAN\n".upper(), fontsize=18, fontweight="bold")

    # PLOTTING
    for j, path in enumerate(dicom_paths):
        image = load_dicom(path)    
        plt.subplot(rows, 8, j+1)
        plt.axis(False)
        plt.imshow(image, cmap="bone")
    plt.tight_layout()
    plt.show()

## **All Scans For Patient <font color="blue">#441</font> - <font color="red">Pulling From Alternative Task 2 Dataset</font>**

In [ ]:
task2_fixed_path = "../input/pngtest/png_voxel_converted_ds/train"

for i, scan_type in enumerate(["FLAIR", "T1w", "T1wCE", "T2w"]):
    png_paths = [os.path.join(task2_fixed_path, img_id, scan_type, x) for x in os.listdir(os.path.join(task2_fixed_path, img_id, scan_type))]
    png_paths = sorted(png_paths, key=lambda x: (int(x.rsplit("-", 1)[-1].split(".", 1)[0])), reverse=True)
    slices = len(png_paths)
    rows = int(np.ceil(slices/8))
    plt.figure(figsize=(20, rows*4))
    plt.suptitle(f"\n\n\n{scan_type} SCAN\n".upper(), fontsize=18, fontweight="bold")

    # PLOTTING
    for j, path in enumerate(png_paths):
        image = cv2.imread(path)
        plt.subplot(rows, 8, j+1)
        plt.axis(False)
        plt.imshow(image, cmap="bone")
    plt.tight_layout()
    plt.show()